In [3]:
from glob import glob
import pandas as pd
import os
import IPython.display as ipd
import numpy as np
import random
import librosa

np.seterr(all='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [5]:
csvs = glob('cv-corpus-5.1-2020-06-22/*/validated.tsv')
len(csvs)

54

In [6]:
os.path.split(csvs[0])

('cv-corpus-5.1-2020-06-22/ia', 'validated.tsv')

In [7]:
# ipd.Audio('cv-corpus-5.1-2020-06-22/ia/clips/common_voice_ia_20911553.mp3')

In [8]:
from tqdm import tqdm

dfs = []
for csv in tqdm(csvs):
    df = pd.read_csv(csv, sep = '\t')
    df = df[df['age'].notnull()]
    df['directory'] = os.path.split(csv)[0]
    dfs.append(df)

 24%|██▍       | 13/54 [00:05<00:27,  1.50it/s]/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 37%|███▋      | 20/54 [00:07<00:14,  2.38it/s]/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 85%|████████▌ | 46/54 [00:12<00:02,  2.78it/s]/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100%|██████████| 54/54 [00:13<00:00,  4.09it/s]


In [9]:
concatenated = pd.concat(dfs, axis = 0)

In [10]:
concatenated['age'].value_counts()

twenties     1084004
thirties      819119
fourties      483789
fifties       423168
teens         203062
sixties       187565
seventies      19490
eighties        4883
nineties        1717
Name: age, dtype: int64

In [11]:
actual_labels = [
    'teens',
    'twenties',
    'thirties',
    'fourties',
    'fifties',
    'sixties',
    'seventies',
    'eighties',
    'nineties',
    'not an age',
]

In [12]:
from scipy.special import expit

def sox_reverb(
    y, reverberance = 1, hf_damping = 1, room_scale = 1, stereo_depth = 1
):
    from pysndfx import AudioEffectsChain

    apply_audio_effects = AudioEffectsChain().reverb(
        reverberance = reverberance,
        hf_damping = hf_damping,
        room_scale = room_scale,
        stereo_depth = stereo_depth,
        pre_delay = 20,
        wet_gain = 0,
        wet_only = False,
    )
    y_enhanced = apply_audio_effects(y)

    return y_enhanced


def sox_augment_low(
    y,
    min_bass_gain = 5,
    reverberance = 1,
    hf_damping = 1,
    room_scale = 1,
    stereo_depth = 1,
    negate = 1,
):
    from pysndfx import AudioEffectsChain

    if negate:
        min_bass_gain = -min_bass_gain
    apply_audio_effects = (
        AudioEffectsChain()
        .lowshelf(gain = min_bass_gain, frequency = 300, slope = 0.1)
        .reverb(
            reverberance = reverberance,
            hf_damping = hf_damping,
            room_scale = room_scale,
            stereo_depth = stereo_depth,
            pre_delay = 20,
            wet_gain = 0,
            wet_only = False,
        )
    )
    y_enhanced = apply_audio_effects(y)

    return y_enhanced


def sox_augment_high(
    y,
    min_bass_gain = 5,
    reverberance = 1,
    hf_damping = 1,
    room_scale = 1,
    stereo_depth = 1,
    negate = 1,
):
    from pysndfx import AudioEffectsChain

    if negate:
        min_bass_gain = -min_bass_gain

    apply_audio_effects = (
        AudioEffectsChain()
        .highshelf(
            gain = -min_bass_gain * (1 - expit(np.max(y))),
            frequency = 300,
            slope = 0.1,
        )
        .reverb(
            reverberance = reverberance,
            hf_damping = hf_damping,
            room_scale = room_scale,
            stereo_depth = stereo_depth,
            pre_delay = 20,
            wet_gain = 0,
            wet_only = False,
        )
    )
    y_enhanced = apply_audio_effects(y)

    return y_enhanced


def sox_augment_combine(
    y,
    min_bass_gain_low = 5,
    min_bass_gain_high = 5,
    reverberance = 1,
    hf_damping = 1,
    room_scale = 1,
    stereo_depth = 1,
):
    from pysndfx import AudioEffectsChain

    apply_audio_effects = (
        AudioEffectsChain()
        .lowshelf(gain = min_bass_gain_low, frequency = 300, slope = 0.1)
        .highshelf(gain = -min_bass_gain_high, frequency = 300, slope = 0.1)
        .reverb(
            reverberance = reverberance,
            hf_damping = hf_damping,
            room_scale = room_scale,
            stereo_depth = stereo_depth,
            pre_delay = 20,
            wet_gain = 0,
            wet_only = False,
        )
    )
    y_enhanced = apply_audio_effects(y)

    return y_enhanced


def random_pitch(sample, low = 0.5, high = 1.0):
    y_pitch_speed = sample.copy()
    length_change = np.random.uniform(low = low, high = high)
    speed_fac = 1.0 / length_change
    tmp = np.interp(
        np.arange(0, len(y_pitch_speed), speed_fac),
        np.arange(0, len(y_pitch_speed)),
        y_pitch_speed,
    )
    minlen = min(y_pitch_speed.shape[0], tmp.shape[0])
    y_pitch_speed *= 0
    y_pitch_speed[:minlen] = tmp[:minlen]
    return y_pitch_speed


def random_amplitude(sample, low = 1.5, high = 3):
    y_aug = sample.copy()
    dyn_change = np.random.uniform(low = low, high = high)
    return y_aug * dyn_change


def random_stretch(sample, low = 0.5, high = 1.3):
    input_length = len(sample)
    stretching = sample.copy()
    random_stretch = np.random.uniform(low = low, high = high)
    stretching = librosa.effects.time_stretch(
        stretching.astype('float'), random_stretch
    )
    return stretching

def add_uniform_noise(sample, power = 0.01):
    y_noise = sample.copy()
    noise_amp = power * np.random.uniform() * np.amax(y_noise)
    return y_noise.astype('float64') + noise_amp * np.random.normal(
        size = y_noise.shape[0]
    )


def add_noise(sample, noise, random_sample = True, factor = 0.1):
    y_noise = sample.copy()
    if len(y_noise) > len(noise):
        noise = np.tile(noise, int(np.ceil(len(y_noise) / len(noise))))
    else:
        if random_sample:
            noise = noise[np.random.randint(0, len(noise) - len(y_noise) + 1) :]
    return y_noise + noise[: len(y_noise)] * factor

def sampling(combined, frame_duration_ms = 700, sample_rate = 16000):
    n = int(sample_rate * (frame_duration_ms / 1000.0))
    offset = 0
    while offset + n <= len(combined):
        yield combined[offset : offset + n]
        offset += n
    if offset < len(combined):
        yield combined[offset:]
        
def calc(signal):

    choice = random.randint(0, 4)
    if choice == 0:

        x = sox_augment_high(
            signal,
            min_bass_gain = random.randint(25, 50),
            reverberance = random.randint(0, 80),
            hf_damping = 10,
            room_scale = random.randint(0, 50),
            negate = 1,
        )
    if choice == 1:
        x = sox_augment_high(
            signal,
            min_bass_gain = random.randint(25, 70),
            reverberance = random.randint(0, 80),
            hf_damping = 10,
            room_scale = random.randint(0, 50),
            negate = 0,
        )
    if choice == 2:
        x = sox_augment_low(
            signal,
            min_bass_gain = random.randint(5, 30),
            reverberance = random.randint(0, 80),
            hf_damping = 10,
            room_scale = random.randint(0, 50),
            negate = random.randint(0, 1),
        )
    if choice == 3:
        x = sox_augment_combine(
            signal,
            min_bass_gain_high = random.randint(25, 70),
            min_bass_gain_low = random.randint(5, 30),
            reverberance = random.randint(0, 80),
            hf_damping = 10,
            room_scale = random.randint(0, 90),
        )
    if choice == 4:
        x = sox_reverb(
            signal,
            reverberance = random.randint(10, 80),
            hf_damping = 10,
            room_scale = random.randint(10, 90),
        )

    if random.randint(0, 1):
        x = add_uniform_noise(
            x, power = random.uniform(0.005, 0.015)
        )
        
    if random.random() > 0.75:
        r = random.choice(not_music)
        n = read_wav(r)[0]
        x = add_noise(x, n, factor = random.uniform(0.005, 0.01))

    return x

In [13]:
os.path.exists('cv-corpus-5.1-2020-06-22/ia/clips/common_voice_ia_20911553.mp3')

True

In [14]:
files, labels = [], []
for label in actual_labels[:-1]:
    print(label)
    d = concatenated[concatenated['age'] == label]
    d = d.sample(n = min(30000, len(d))).values
    for i in tqdm(range(len(d))):
        f = f"{d[i,-1]}/clips/{d[i,1]}"
        files.append(f)
        labels.append(d[i, -6])

teens


100%|██████████| 30000/30000 [00:00<00:00, 764574.72it/s]


twenties


100%|██████████| 30000/30000 [00:00<00:00, 467316.30it/s]


thirties


...

100%|██████████| 30000/30000 [00:00<00:00, 468651.28it/s]


seventies


100%|██████████| 4883/4883 [00:00<00:00, 478813.92it/s]


eighties
nineties


100%|██████████| 1717/1717 [00:00<00:00, 504068.03it/s]


In [15]:
from scipy import interpolate
from pydub import AudioSegment
import soundfile as sf

def int_to_float(array, type = np.float32):

    if array.dtype == type:
        return array

    if array.dtype not in [np.float16, np.float32, np.float64]:
        array = array.astype(np.float32) / np.max(np.abs(array))

    return array

def change_samplerate(data, old_samplerate, new_samplerate):
    old_audio = data
    duration = data.shape[0] / old_samplerate
    time_old = np.linspace(0, duration, old_audio.shape[0])
    time_new = np.linspace(
        0, duration, int(old_audio.shape[0] * new_samplerate / old_samplerate)
    )

    interpolator = interpolate.interp1d(time_old, old_audio.T)
    data = interpolator(time_new).T
    return data


def read_flac(file, sample_rate = 16000):
    data, old_samplerate = sf.read(file)
    if len(data.shape) == 2:
        data = data[:, 0]
    if old_samplerate != sample_rate:
        data = change_samplerate(data, old_samplerate, sample_rate)
    return data, sample_rate


def read_wav(file, sample_rate = 16000):
    y, sr = librosa.load(file, sr = sample_rate)
    return y, sr

def read_mp3(file, sample_rate = 16000):
    audio = AudioSegment.from_mp3(file)
    a = np.array(audio.set_frame_rate(sample_rate).set_channels(1).get_array_of_samples())
    return int_to_float(a), sample_rate

def read_file(file):
    if '.flac' in file or '.wav' in file:
        y, sr = read_flac(file)
    if '.mp3' in file:
        y, sr = read_mp3(file)
    return y, sr

In [16]:
np.unique(labels, return_counts = True)

(array(['eighties', 'fifties', 'fourties', 'nineties', 'seventies',
        'sixties', 'teens', 'thirties', 'twenties'], dtype='<U9'),
 array([ 4883, 30000, 30000,  1717, 19490, 30000, 30000, 30000, 30000]))

In [17]:
not_music = glob('../not-music/clean-wav/*.wav') + glob('../musan/music/**/*.wav', recursive = True) \
+ glob('../musan/noise/**/*.wav', recursive = True)
files = files + not_music
labels = labels + ['not an age'] * len(not_music)

In [18]:
np.unique(labels, return_counts = True)

(array(['eighties', 'fifties', 'fourties', 'nineties', 'not an age',
        'seventies', 'sixties', 'teens', 'thirties', 'twenties'],
       dtype='<U10'),
 array([ 4883, 30000, 30000,  1717,  2026, 19490, 30000, 30000, 30000,
        30000]))

In [19]:
from sklearn.utils import shuffle
files, labels = shuffle(files, labels)

In [20]:
y, sr = read_file(files[-2])
y = y[:sr * 10]
y, sr, labels[-2]

(array([0.000000e+00, 0.000000e+00, 0.000000e+00, ..., 3.051851e-05,
        0.000000e+00, 0.000000e+00], dtype=float32),
 16000,
 'fifties')

In [21]:
import os
import tensorflow as tf

os.system('rm age-detection/data/*')
DATA_DIR = os.path.expanduser('age-detection/data')
tf.gfile.MakeDirs(DATA_DIR)

In [22]:
import malaya_speech

vad = malaya_speech.vad.webrtc()

In [23]:
from tqdm import tqdm
from malaya_speech.train import prepare_data
from collections import defaultdict

def loop(files, dupe_factor = 1):
    files, no = files
    fname = f'{DATA_DIR}/part-{no}.tfrecords'
    writer = tf.python_io.TFRecordWriter(fname)
    counts = defaultdict(int)
    for file in tqdm(files):
        try:
            wav = read_file(file[0])[0]
            if file[1] != 'not an age':
                d = dupe_factor
            else:
                d = 1
            for _ in range(d):
                if file[1] != 'not an age':
                    minimum = 1000
                else:
                    minimum = 200
                fs = sampling(wav, random.randint(minimum, 2000))
                for s in fs:
                    try:
                        if file[1] != 'not an age':
                            n = malaya_speech.utils.astype.float_to_int(s)
                            frames = malaya_speech.utils.generator.frames(n, 30, 16000, append_ending_trail=False)
                            frames = [f.array for f in frames if vad(f)]
                            n = malaya_speech.utils.astype.int_to_float(np.concatenate(frames))
                            for _ in range(dupe_factor):
                                if len(n) > 50:
                                    example = prepare_data.to_example({'inputs': calc(n).tolist(), 
                                                                       'targets': [actual_labels.index(file[1])]})
                                    writer.write(example.SerializeToString())
                                    counts[file[1]] += 1
                        else:
                            n = s
                        if len(n) > 50:
                            example = prepare_data.to_example({'inputs': n.tolist(), 
                                                               'targets': [actual_labels.index(file[1])]})
                            writer.write(example.SerializeToString())
                            counts[file[1]] += 1
                    except Exception as e:
                        # print(e)
                        pass
        except Exception as e:
            print(e)
            pass

    writer.close()
    return [counts]

In [24]:
combined_all = list(zip(files, labels))

In [25]:
combined_all[:10]

[('cv-corpus-5.1-2020-06-22/en/clips/common_voice_en_19028924.mp3',
  'thirties'),
 ('cv-corpus-5.1-2020-06-22/fr/clips/common_voice_fr_18312975.mp3',
  'fourties'),
 ('cv-corpus-5.1-2020-06-22/el/clips/common_voice_el_20583935.mp3',
  'fourties'),
 ('cv-corpus-5.1-2020-06-22/en/clips/common_voice_en_18887255.mp3', 'sixties'),
 ('cv-corpus-5.1-2020-06-22/de/clips/common_voice_de_18357436.mp3', 'teens'),
 ('cv-corpus-5.1-2020-06-22/en/clips/common_voice_en_22299580.mp3',
  'seventies'),
 ('cv-corpus-5.1-2020-06-22/de/clips/common_voice_de_20363556.mp3',
  'fourties'),
 ('cv-corpus-5.1-2020-06-22/rw/clips/common_voice_rw_20337788.mp3', 'teens'),
 ('cv-corpus-5.1-2020-06-22/fa/clips/common_voice_fa_19567988.mp3',
  'twenties'),
 ('cv-corpus-5.1-2020-06-22/en/clips/common_voice_en_21640941.mp3',
  'thirties')]

In [26]:
# !cp ../mp.py .

In [27]:
import mp
returned = mp.multiprocessing(combined_all, loop, cores = 20)

 86%|████████▋ | 8979/10405 [6:45:25<1:04:14,  2.70s/it] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 91%|█████████ | 9422/10405 [7:02:31<41:19,  2.52s/it]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 94%|█████████▍| 9820/10405 [7:20:17<23:50,  2.44s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--Not

In [28]:
combined_d = defaultdict(int)
for d in returned:
    for k, v in d.items():
        combined_d[k] += v
combined_d

defaultdict(int,
            {'thirties': 193428,
             'fourties': 218722,
             'sixties': 225060,
             'teens': 197550,
             'seventies': 151264,
             'twenties': 205174,
             'not an age': 496484,
             'fifties': 207638,
             'eighties': 39850,
             'nineties': 12314})